In [2]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
import numpy as np

df = pd.read_csv('DataCoSupplyChainDataset.csv', encoding='latin-1')

df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
    .str.replace('(', '')
    .str.replace(')', '')
)

cols_to_drop = ['customer_email', 'customer_password', 'product_image']
df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

date_cols = [col for col in df.columns if 'date' in col]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

df['shipping_delay_days'] = df['days_for_shipping_real'] - df['days_for_shipment_scheduled']
df['order_month'] = df['order_date_dateorders'].dt.to_period('M')
df['order_year'] = df['order_date_dateorders'].dt.year
df['order_day_of_week'] = df['order_date_dateorders'].dt.day_name()
df['profit_margin_pct'] = np.where(
    df['sales'] != 0,
    (df['order_profit_per_order'] / df['sales'] * 100).round(2),
    0
)

print(f"Loaded and cleaned: {df.shape[0]:,} rows x {df.shape[1]} columns")

Saving DataCoSupplyChainDataset.csv to DataCoSupplyChainDataset (1).csv
Saving DescriptionDataCoSupplyChain.csv to DescriptionDataCoSupplyChain (1).csv
Saving tokenized_access_logs.csv to tokenized_access_logs (1).csv
Loaded and cleaned: 180,519 rows x 55 columns


In [4]:
import sqlite3

df['order_month'] = df['order_month'].astype(str)

conn = sqlite3.connect('supply_chain.db')
df.to_sql('orders', conn, if_exists='replace', index=False)

test = pd.read_sql("SELECT COUNT(*) as total_rows FROM orders", conn)
print(f"Database created with {test['total_rows'][0]:,} rows")

Database created with 180,519 rows


In [5]:
query1 = """
SELECT
    shipping_mode,
    COUNT(*) AS total_orders,
    SUM(CASE WHEN late_delivery_risk = 0 THEN 1 ELSE 0 END) AS on_time_orders,
    ROUND(
        SUM(CASE WHEN late_delivery_risk = 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
        2
    ) AS on_time_rate_pct
FROM orders
GROUP BY shipping_mode
ORDER BY on_time_rate_pct DESC
"""

kpi1 = pd.read_sql(query1, conn)
print("KPI 1: On-Time Delivery Rate by Shipping Mode")
kpi1

KPI 1: On-Time Delivery Rate by Shipping Mode


,shipping_mode,total_orders,on_time_orders,on_time_rate_pct
0,Standard Class,107752,66729,61.93
1,Same Day,9737,5283,54.26
2,Second Class,35216,8229,23.37
3,First Class,27814,1301,4.68


In [6]:
query2 = """
SELECT
    market,
    COUNT(*) AS total_orders,
    ROUND(AVG(shipping_delay_days), 2) AS avg_delay_days,
    ROUND(AVG(days_for_shipping_real), 2) AS avg_actual_days,
    ROUND(AVG(days_for_shipment_scheduled), 2) AS avg_scheduled_days
FROM orders
GROUP BY market
ORDER BY avg_delay_days DESC
"""

kpi2 = pd.read_sql(query2, conn)
print("KPI 2: Average Shipping Delay by Market Region")
kpi2

KPI 2: Average Shipping Delay by Market Region


,market,total_orders,avg_delay_days,avg_actual_days,avg_scheduled_days
0,USCA,25799,0.57,3.48,2.92
1,Pacific Asia,41260,0.57,3.50,2.93
2,Europe,50252,0.57,3.49,2.92
3,LATAM,51594,0.56,3.51,2.95
4,Africa,11614,0.56,3.51,2.95


In [7]:
query3 = """
SELECT
    category_name,
    COUNT(*) AS total_orders,
    ROUND(SUM(sales), 2) AS total_revenue,
    ROUND(SUM(order_profit_per_order), 2) AS total_profit,
    ROUND(AVG(profit_margin_pct), 2) AS avg_margin_pct
FROM orders
GROUP BY category_name
ORDER BY total_revenue DESC
LIMIT 15
"""

kpi3 = pd.read_sql(query3, conn)
print("KPI 3: Revenue & Profit by Category")
kpi3

KPI 3: Revenue & Profit by Category


,category_name,total_orders,total_revenue,total_profit,avg_margin_pct
0,Fishing,17325,6929653.69,756220.77,10.91
1,Cleats,24551,4431942.78,494636.92,11.19
2,Camping & Hiking,13729,4118425.57,427455.57,10.38
3,Cardio Equipment,12487,3694843.20,383011.10,10.62
4,Women's Apparel,21035,3147800.00,350421.03,11.00
5,Water Sports,15540,3113844.68,325146.96,10.45
6,Men's Footwear,22246,2891757.66,311902.82,10.79
7,Indoor/Outdoor Games,19298,2888993.91,318451.43,11.13
8,Shop By Sport,10984,1309522.04,129813.96,10.09
9,Computers,442,663000.00,69656.81,10.51


In [8]:
query4 = """
SELECT
    CASE
        WHEN order_count = 1 THEN '1 - One-Time'
        WHEN order_count BETWEEN 2 AND 5 THEN '2 - Occasional'
        WHEN order_count BETWEEN 6 AND 15 THEN '3 - Regular'
        ELSE '4 - VIP'
    END AS customer_segment,
    COUNT(*) AS customer_count,
    ROUND(AVG(total_spend), 2) AS avg_lifetime_spend,
    ROUND(AVG(avg_order_value), 2) AS avg_order_value
FROM (
    SELECT
        customer_id,
        COUNT(*) AS order_count,
        SUM(sales) AS total_spend,
        AVG(sales) AS avg_order_value
    FROM orders
    GROUP BY customer_id
)
GROUP BY customer_segment
ORDER BY customer_segment
"""

kpi4 = pd.read_sql(query4, conn)
print("KPI 4: Customer Segmentation")
kpi4

KPI 4: Customer Segmentation


,customer_segment,customer_count,avg_lifetime_spend,avg_order_value
0,1 - One-Time,8435,294.51,294.51
1,2 - Occasional,1173,783.25,202.20
2,3 - Regular,6393,2138.76,199.25
3,4 - VIP,4651,4237.52,199.38


In [9]:
query5 = """
SELECT
    strftime('%Y-%m', order_date_dateorders) AS order_month,
    COUNT(*) AS order_count,
    ROUND(SUM(sales), 2) AS monthly_revenue,
    ROUND(SUM(order_profit_per_order), 2) AS monthly_profit,
    ROUND(SUM(CASE WHEN late_delivery_risk = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS late_pct
FROM orders
GROUP BY order_month
ORDER BY order_month
"""

kpi5 = pd.read_sql(query5, conn)
print("KPI 5: Monthly Revenue Trend")
kpi5

KPI 5: Monthly Revenue Trend


,order_month,order_count,monthly_revenue,monthly_profit,late_pct
0,2015-01,5322,1051590.08,111660.74,54.11
1,2015-02,4729,927009.90,99140.66,54.85
2,2015-03,5362,1051253.69,113778.21,54.76
3,2015-04,5126,1014463.28,108083.68,53.84
4,2015-05,5357,1050478.44,112147.90,55.09
5,2015-06,5134,1024006.17,110147.16,54.11
6,2015-07,5299,1038081.19,115624.06,55.46
7,2015-08,5273,1029494.69,117979.77,55.68
8,2015-09,5140,1018338.60,113467.94,56.69
9,2015-10,5302,1049154.27,101757.87,54.81


In [11]:
query6 = """
SELECT
    market,
    category_name,
    COUNT(*) AS total_orders,
    SUM(CASE WHEN order_status = 'SUSPECTED_FRAUD' THEN 1 ELSE 0 END) AS fraud_count,
    ROUND(
        SUM(CASE WHEN order_status = 'SUSPECTED_FRAUD' THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
        2
    ) AS fraud_rate_pct
FROM orders
GROUP BY market, category_name
HAVING fraud_count > 0
ORDER BY fraud_rate_pct DESC
LIMIT 20
"""

kpi6 = pd.read_sql(query6, conn)
print("KPI 6: Top 20 Fraud Hotspots by Market & Category")
kpi6

KPI 6: Top 20 Fraud Hotspots by Market & Category


,market,category_name,total_orders,fraud_count,fraud_rate_pct
0,Pacific Asia,Books,97,8,8.25
1,USCA,Lacrosse,46,3,6.52
2,Pacific Asia,Boxing & MMA,72,4,5.56
3,USCA,Boxing & MMA,54,3,5.56
4,Africa,Girls' Apparel,80,4,5.00
5,Pacific Asia,Fitness Accessories,82,4,4.88
6,Europe,Women's Golf Clubs,107,5,4.67
7,USCA,Trade-In,156,7,4.49
8,Europe,Basketball,67,3,4.48
9,Africa,Golf Balls,120,5,4.17


In [12]:
query7 = """
SELECT
    shipping_mode,
    ROUND(AVG(days_for_shipping_real), 2) AS avg_delivery_days,
    ROUND(AVG(sales), 2) AS avg_order_value,
    ROUND(AVG(order_profit_per_order), 2) AS avg_profit,
    ROUND(AVG(benefit_per_order), 2) AS avg_benefit,
    COUNT(*) AS total_orders
FROM orders
GROUP BY shipping_mode
ORDER BY avg_delivery_days
"""

kpi7 = pd.read_sql(query7, conn)
print("KPI 7: Shipping Mode Performance Comparison")
kpi7

KPI 7: Shipping Mode Performance Comparison


,shipping_mode,avg_delivery_days,avg_order_value,avg_profit,avg_benefit,total_orders
0,Same Day,0.48,199.50,20.85,20.85,9737
1,First Class,2.00,204.01,23.12,23.12,27814
2,Second Class,3.99,202.90,21.31,21.31,35216
3,Standard Class,4.00,204.38,22.00,22.00,107752


In [13]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [14]:
fig = px.bar(
    kpi1,
    x='shipping_mode',
    y='on_time_rate_pct',
    color='on_time_rate_pct',
    color_continuous_scale='RdYlGn',
    text='on_time_rate_pct',
    title='On-Time Delivery Rate by Shipping Mode'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(yaxis_title='On-Time Rate (%)', xaxis_title='Shipping Mode')
fig.show()

In [15]:
fig = px.bar(
    kpi2,
    x='market',
    y='avg_delay_days',
    color='avg_delay_days',
    color_continuous_scale='OrRd',
    text='avg_delay_days',
    title='Average Shipping Delay by Market Region'
)
fig.update_traces(texttemplate='%{text:.2f} days', textposition='outside')
fig.update_layout(yaxis_title='Avg Delay (Days)', xaxis_title='Market')
fig.show()

In [16]:
fig = go.Figure()
fig.add_trace(go.Bar(
    name='Revenue',
    x=kpi3['category_name'],
    y=kpi3['total_revenue'],
    marker_color='#2196F3'
))
fig.add_trace(go.Bar(
    name='Profit',
    x=kpi3['category_name'],
    y=kpi3['total_profit'],
    marker_color='#4CAF50'
))
fig.update_layout(
    barmode='group',
    title='Revenue vs Profit — Top 15 Product Categories',
    xaxis_tickangle=-45,
    yaxis_title='Amount ($)',
    xaxis_title='Category'
)
fig.show()

In [17]:
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "bar"}]],
                    subplot_titles=("Customer Count by Segment", "Avg Lifetime Spend by Segment"))

fig.add_trace(go.Pie(
    labels=kpi4['customer_segment'],
    values=kpi4['customer_count'],
    hole=0.4
), row=1, col=1)

fig.add_trace(go.Bar(
    x=kpi4['customer_segment'],
    y=kpi4['avg_lifetime_spend'],
    marker_color=['#FF7043', '#FFA726', '#66BB6A', '#42A5F5'],
    text=kpi4['avg_lifetime_spend'].apply(lambda x: f'${x:,.0f}'),
    textposition='outside'
), row=1, col=2)

fig.update_layout(title_text='Customer Segmentation Analysis', showlegend=True, height=500)
fig.show()

In [18]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(
    x=kpi5['order_month'],
    y=kpi5['monthly_revenue'],
    name='Monthly Revenue',
    line=dict(color='#2196F3', width=2),
    fill='tozeroy',
    fillcolor='rgba(33,150,243,0.1)'
), secondary_y=False)

fig.add_trace(go.Scatter(
    x=kpi5['order_month'],
    y=kpi5['late_pct'],
    name='Late Delivery %',
    line=dict(color='#F44336', width=2, dash='dot')
), secondary_y=True)

fig.update_layout(title='Monthly Revenue vs Late Delivery Rate')
fig.update_yaxes(title_text='Revenue ($)', secondary_y=False)
fig.update_yaxes(title_text='Late Delivery %', secondary_y=True)
fig.show()

In [19]:
fraud_pivot = kpi6.pivot_table(index='market', columns='category_name', values='fraud_rate_pct', fill_value=0)

fig = px.imshow(
    fraud_pivot,
    color_continuous_scale='YlOrRd',
    title='Fraud Rate Heatmap — Market x Category',
    labels=dict(color='Fraud Rate %')
)
fig.update_layout(height=500, xaxis_tickangle=-45)
fig.show()

---

## Deliverable 2 — Business Recommendations

### Shipping Mode Strategy
First Class shipping has the lowest on-time rate despite being a premium tier. Recommend auditing the carrier contracts for First Class and either renegotiating SLAs or rerouting First Class orders through a higher-performing carrier. Same Day shipping should be promoted for high-value orders where margin can absorb the cost.

### Regional Focus
Prioritize delivery improvements in Europe and LATAM — these markets carry the highest revenue concentration and any delivery issues have amplified financial impact. Consider regional distribution centers to reduce average shipping times.

### Product Category Profitability
Categories with high revenue but low or negative profit margins need pricing reviews. Implement minimum margin thresholds on discounts to prevent orders from going below breakeven.

### Customer Retention
VIP customers (6+ orders) generate significantly higher lifetime spend. Build a loyalty program targeting Occasional customers (2-5 orders) to move them into Regular/VIP tiers through targeted offers on high-margin categories.

### Fraud Prevention
Certain market-category combinations show elevated fraud rates. Implement order review triggers for orders matching these high-risk profiles — particularly high-value orders from flagged regions — before shipping.